In [39]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import copy
from DS import Queue, PriorityQueue, Stack
import numpy as np

First, let us read in the data from the files romania_map.csv and romania_loc.csv

In [34]:
import networkx as nx # for graph manipulations


# read in the graph of cities and their traversal costs
map_graph = nx.Graph()
with open('romania_map.csv', 'r') as fp:
    lines = list(fp)
    lines = lines[1:] # throw away header line
    lines = map(lambda x: x.strip().split(','), lines) # split each line by commas
    for frm, to, cost in lines:
        map_graph.add_edge(u_of_edge=frm, v_of_edge=to, cost=float(cost))
        


In [35]:
map_graph.nodes()

NodeView(('Arad', 'Zerind', 'Sibiu', 'Timisoara', 'Bucharest', 'Urziceni', 'Pitesti', 'Giurgiu', 'Fagaras', 'Craiova', 'Drobeta', 'Rimnicu', 'Mehadia', 'Eforie', 'Hirsova', 'Iasi', 'Vaslui', 'Neamt', 'Lugoj', 'Oradea'))

In [36]:
map_graph.edges(nbunch='Arad', data='cost')

EdgeDataView([('Arad', 'Zerind', 75.0), ('Arad', 'Sibiu', 140.0), ('Arad', 'Timisoara', 118.0)])

Read in coordinate information for heuristics

In [37]:
coords = {}

with open('romania_loc.csv', 'r') as fp:
    lines = list(fp)
    lines = lines[1:]
    lines = map(lambda x: x.strip().split(','), lines) # split each line by commas
    for loc, x, y in lines:
        coords[loc] = (float(x), float(y))
        
def distance_between(city1, city2):
    x1, y1 = coords[city1]
    x2, y2 = coords[city2]
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    
    

In [38]:
class State(object):
    def __init__(self, city):
        self.city = city
        self.f = 0 # approx cost from start to goal through this state
        self.g = 0 # actual cost from start
        self.h = 0 # approx cost from goal
    
    
    # give string representation for easier display
    def __str__(self):
        return self.city
    
    # used to provide hashcode for hashing-based data structures like dictionaries and sets
    def __hash__(self):
        return hash(self.city)
    
    # need to give dummy implementation for priority queue based algorithms
    def __lt__(self, other):
        return 0

In [46]:
class Graph(object):
    def __init__(self, solution_state):
        self.solution_state = State(solution_state)
        
    # admissable heuristic on states
    def h(self, state):
        return distance_between(state.city, self.solution_state.city)
    
    # generate states that can be reached from another state 
    def get_edges(self, state):
        for _, adj_city, cost in map_graph.edges(nbunch=state.city, data='cost'):
            adj_state = State(adj_city)
            yield adj_state, cost

In [47]:
graph = Graph('Bucharest')

In [48]:
print(graph.solution_state)
for s, i in graph.get_edges(graph.solution_state):
    print(s, ' ', i)

Bucharest
Urziceni   85.0
Pitesti   101.0
Giurgiu   90.0
Fagaras   211.0


In [49]:
def print_solution(parents, goal):
    curr = goal
    stack = Stack()
    stack.push(curr)
    while parents[curr]:
        curr = parents[curr]
        stack.push(curr)
    while stack:
        print(stack.pop())
        
def bfs(start, goal_test, graph):
    fronteir = Queue()
    fronteir.enqueue(start)
    explored = set()
    parents = {start: None}
    while fronteir:
        current = fronteir.dequeue()
        explored.add(current)
        if goal_test(current):
            print_solution(parents, current)
            return True
        for adj_state, cost in graph.get_edges(current):
            if adj_state not in fronteir and adj_state not in explored:
                fronteir.enqueue(adj_state)
                parents[adj_state] = current
    print('No Solution found')
    return False
        
    

def a_star(start, goal_test, graph):
    pqueue = PriorityQueue()
    pqueue.push(0, start)
    parents = {start: None}
    explored = set()
    while pqueue:
        i, current = pqueue.pop()
        explored.add(current)
        if goal_test(current):
            print_solution(parents, current)
            return True
        for adj_state, cost in graph.get_edges(current):
            g = current.g + cost
            h = graph.h(adj_state)
            f = g + h
            if (adj_state in pqueue and pqueue[adj_state] > f) or adj_state not in pqueue:
                adj_state.f = f
                adj_state.g = g
                adj_state.h = h
                pqueue[adj_state] = f
                parents[adj_state] = current
    print('No Solution found')
    return False
            
            
        

In [50]:
def goal_test(state):
    return state.city == 'Bucharest'

start_state = State('Arad')

In [51]:
bfs(start_state, goal_test, graph)

Arad
Sibiu
Fagaras
Bucharest


True

In [52]:
a_star(start_state, goal_test, graph)

TypeError: unsupported operand type(s) for -: 'str' and 'str'